In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
'''
!pip install python-dotenv
!pip install neo4j
!pip install pandas
!pip install langchain
!pip install transformers
!pip install langchain.chains
'''

# Note: will do a requirements.txt file later

'\n!pip install python-dotenv\n!pip install neo4j\n!pip install pandas\n!pip install langchain\n!pip install transformers\n!pip install langchain.chains\n'

In [3]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
import pandas as pd
from xlsx_to_csv import *
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

URI = NEO4J_URI
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [4]:
from kg import *

create_kg()

Knowledge Graph created with 33 nodes and 96 relationships.


In [5]:
def query_knowledge_graph(query):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (n:Person)-[r]->(m:Person)
            WHERE n.name CONTAINS $query OR m.name CONTAINS $query
            RETURN n.name, r, m.name
            """,
            query=query
        )
        return result.data()


In [6]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

graph.refresh_schema()

print(graph.schema)

Node properties:
Person {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:TEAMMATE]->(:Person)
(:Person)-[:FRIEND]->(:Person)
(:Person)-[:CLASSMATE]->(:Person)
(:Person)-[:FAMILY]->(:Person)
(:Person)-[:IMMEDIATE_FAMILY]->(:Person)
(:Person)-[:LOVER]->(:Person)
(:Person)-[:OPPONENT]->(:Person)
(:Person)-[:ENEMY]->(:Person)
(:Person)-[:TEACHER]->(:Person)


In [12]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({'query': 'List all of the characters who are not friends with Harry'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person) WHERE p.name <> 'Harry' AND NOT (p)-[:FRIEND]->(:Person {name: 'Harry'}) RETURN p

Full Context:
[{'p': {'name': 'Piers'}}, {'p': {'name': 'James'}}, {'p': {'name': 'Lily'}}, {'p': {'name': 'Petunia'}}, {'p': {'name': 'Vernon Dursley'}}, {'p': {'name': 'Dudley'}}, {'p': {'name': 'Dumbledore'}}, {'p': {'name': 'McGonagall'}}, {'p': {'name': 'Snape'}}, {'p': {'name': 'Charlie'}}]

> Finished chain.


{'query': 'List all of the characters who are not friends with Harry',
 'result': 'The characters who are not friends with Harry are Piers, James, Lily, Petunia, Vernon Dursley, Dudley, Dumbledore, McGonagall, Snape, and Charlie.'}

In [8]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY')
)
    
models = client.models.list()

models

SyncPage[Model](data=[Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='text-embedding-3-small', created=1705948997, object='model', owned_by='system'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', 